In [21]:
import itertools

import click
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_ranking as tfr
from sklearn.metrics import f1_score

import logging
logger = logging.getLogger(__name__)


tf.enable_eager_execution()
tf.executing_eagerly()

# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

_TRAIN_DATA_PATH = ''
_TEST_DATA_PATH = ''

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS = "pairwise_logistic_loss"
# _LOSS = "sigmoid_cross_entropy_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE = 100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.

# Parameters to the scoring function.
_BATCH_SIZE = 100
_HIDDEN_LAYER_DIMS = ["20", "10"]


# _OUT_DIR = "../models/tfranking/"

def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
        tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
        output_types=(
            {str(k): tf.float32 for k in range(1, _NUM_FEATURES + 1)},
            tf.float32
        ),
        output_shapes=(
            {str(k): tf.TensorShape([_LIST_SIZE, 1])
             for k in range(1, _NUM_FEATURES + 1)},
            tf.TensorShape([_LIST_SIZE])
        )
    )

    train_dataset = train_dataset.batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()


def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
        "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
        name: tf.feature_column.numeric_column(
            name, shape=(1,), default_value=0.0) for name in feature_names
    }


def make_score_fn():
    """Returns a scoring function to build `EstimatorSpec`."""

    def _score_fn(context_features, group_features, mode, params, config):
        """Defines the network to score a documents."""
        del params
        del config
        # Define input layer.
        example_input = [
            tf.layers.flatten(group_features[name])
            for name in sorted(example_feature_columns())
        ]
        input_layer = tf.concat(example_input, 1)

        cur_layer = input_layer
        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.layers.dense(
                cur_layer,
                units=layer_width,
                activation="tanh")

        logits = tf.layers.dense(cur_layer, units=1)
        return logits

    return _score_fn


def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
        "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
            tfr.metrics.RankingMetricKey.NDCG, topn=topn)
        for topn in [1, 3, 5, 10]
    })

    return metric_fns


def get_estimator(hparams):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """

    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
        loss_fn=tfr.losses.make_loss_fn(_LOSS),
        eval_metric_fns=eval_metric_fns(),
        train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
        model_fn=tfr.model.make_groupwise_ranking_fn(
            group_score_fn=make_score_fn(),
            group_size=1,
            transform_fn=None,
            ranking_head=ranking_head),
        params=hparams)


def ltr_to_submission(df, features, ranker, path):
    features = features + ['sid']

    preds = ranker.predict(input_fn=lambda: input_fn(path))
    import itertools
    import numpy as np
    # Not sure how to get all preds because it runs infinit
    # So I take all till list size
    preds_slice = itertools.islice(preds, len(df))
    count = 0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count] = i
        count += 1

    test_X = df[features]

    test_X = test_X.assign(yhat=a[:, 0])

    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
        'yhat', 'transport_mode'
    ]]

    from sklearn.metrics import f1_score
    score = f1_score(df.groupby("sid").first()['click_mode'], df_end.transport_mode, average='weighted')
    print('F1 Score is: {}'.format(score))

    return df_end


In [48]:
df_train_train = pd.read_pickle("../data/processed/ranking/train_all_row_sample_50.pickle")
df_train_test = pd.read_pickle("../data/processed/ranking/train_all_row_sample_50.pickle")

_TRAIN_DATA_PATH="../data/processed/ranking/train_all_row_sample_50.libsvm"
_TEST_DATA_PATH="../data/processed/ranking/train_all_row_sample_50.libsvm"

with open('../data/processed/ranking/features_tfranking.txt') as f:
    features = f.read().splitlines()

_NUM_FEATURES = len(features)

hparams = tf.contrib.training.HParams(learning_rate=0.001)
ranker = get_estimator(hparams)

ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

df_preds = ltr_to_submission(df_train_test, features, ranker, _TEST_DATA_PATH)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpiyyzwcdf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd674549c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpo

/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 67.687485, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 67.687485.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score is: 0.5933333333333333


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
i = 0
while i < 10000:
    ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100000)
    i +=1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-101
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 101 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 57.665962, step = 102
INFO:tensorflow:Saving checkpoints for 102 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 57.665962.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-102
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 102 into /tmp/tm

INFO:tensorflow:Saving checkpoints for 115 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 57.02659.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-115
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 115 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 56.979057, step = 116
INFO:tensorflow:Saving checkpoints for 116 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 56.979057.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-116
INFO:tensorflo

INFO:tensorflow:Saving checkpoints for 128 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 56.380344, step = 129
INFO:tensorflow:Saving checkpoints for 129 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 56.380344.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-129
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 129 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 56.33569, step = 130
INFO:tensorflow:Saving checkpoints for 130 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 56.33569.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 142 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 55.771782, step = 143
INFO:tensorflow:Saving checkpoints for 143 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 55.771782.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-143
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 143 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 55.729614, step = 144
INFO:tensorflow:Saving checkpoints for 144 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 55.729614.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
IN

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-156
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 156 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 55.19609, step = 157
INFO:tensorflow:Saving checkpoints for 157 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 55.19609.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-157
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 157 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 55.156124, step = 158
INFO:tensorflow:Saving checkpoints for 158 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorfl

...

INFO:tensorflow:Loss for final step: 34.01707.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2317
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2317 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 34.014378, step = 2318
INFO:tensorflow:Saving checkpoints for 2318 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 34.014378.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2318
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INF

INFO:tensorflow:loss = 33.97975, step = 2331
INFO:tensorflow:Saving checkpoints for 2331 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 33.97975.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2331
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2331 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 33.977108, step = 2332
INFO:tensorflow:Saving checkpoints for 2332 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 33.977108.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters f

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2344 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 33.943, step = 2345
INFO:tensorflow:Saving checkpoints for 2345 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 33.943.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2345
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2345 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 33.940395, step = 2346
INFO:tensorflow:Saving checkpoints for 2346 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 33.940395.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
I

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2358
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2358 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 33.906773, step = 2359
INFO:tensorflow:Saving checkpoints for 2359 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:Loss for final step: 33.906773.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2359
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2359 into /tmp/tmpiyyzwcdf/model.ckpt.
INFO:tensorflow:loss = 33.904205, step = 2360
INFO:tensorflow:Saving checkpoints fo

INFO:tensorflow:Loss for final step: 33.873573.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiyyzwcdf/model.ckpt-2372
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2372 into /tmp/tmpiyyzwcdf/model.ckpt.


# 100 k

In [41]:
df_train_train = pd.read_pickle("../data/processed/ranking/train_all_row_sample_100k.pickle")
df_train_test = pd.read_pickle("../data/processed/ranking/train_all_row_sample_100k.pickle")

_TRAIN_DATA_PATH="../data/processed/ranking/train_all_row_sample_100k.libsvm"
_TEST_DATA_PATH="../data/processed/ranking/train_all_row_sample_100k.libsvm"

with open('../data/processed/ranking/features_tfranking.txt') as f:
    features = f.read().splitlines()

_NUM_FEATURES = len(features)

hparams = tf.contrib.training.HParams(learning_rate=0.001)
ranker = get_estimator(hparams)

ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100000)

df_preds = ltr_to_submission(df_train_test, features, ranker, _TEST_DATA_PATH)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6u9m8hpf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5f40de8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpo

/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp6u9m8hpf/model.ckpt.
INFO:tensorflow:loss = 68.177574, step = 1
INFO:tensorflow:global_step/sec: 14.8009
INFO:tensorflow:loss = 64.45798, step = 101 (6.758 sec)
INFO:tensorflow:global_step/sec: 13.6414
INFO:tensorflow:loss = 64.24874, step = 201 (7.330 sec)
INFO:tensorflow:global_step/sec: 14.9547
INFO:tensorflow:loss = 65.40012, step = 301 (6.687 sec)
INFO:tensorflow:global_step/sec: 14.3119
INFO:tensorflow:loss = 63.03412, step = 401 (6.987 sec)
INFO:tensorflow:global_step/sec: 10.8212
INFO:tensorflow:loss = 62.565254, step = 501 (9.241 sec)
INFO:tensorflow:global_step/sec: 11.9035
INFO:tensorflow:loss = 61.585995, step = 601 (8.401 sec)
INFO:tensorflow:global_step/sec: 12.6482
INFO:tensorflow:loss = 63.88342, step = 701 (7.907 sec)
INFO:tensorflow:global_step/sec: 9.53363
INFO:tensorflow:loss = 63.386856, step =

# 1k 

In [42]:
df_train_train = pd.read_pickle("../data/processed/ranking/train_all_row_sample_1k.pickle")
df_train_test = pd.read_pickle("../data/processed/ranking/train_all_row_sample_1k.pickle")

_TRAIN_DATA_PATH="../data/processed/ranking/train_all_row_sample_1k.libsvm"
_TEST_DATA_PATH="../data/processed/ranking/train_all_row_sample_1k.libsvm"

with open('../data/processed/ranking/features_tfranking.txt') as f:
    features = f.read().splitlines()

_NUM_FEATURES = len(features)

hparams = tf.contrib.training.HParams(learning_rate=0.001)
ranker = get_estimator(hparams)

ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100000)

df_preds = ltr_to_submission(df_train_test, features, ranker, _TEST_DATA_PATH)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpw1w9j_7b', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5f807d160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkpo

/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpw1w9j_7b/model.ckpt.
INFO:tensorflow:loss = 68.24118, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /tmp/tmpw1w9j_7b/model.ckpt.
INFO:tensorflow:Loss for final step: 67.33364.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpw1w9j_7b/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score is: 0.32420710379700923


In [47]:
i = 0
while i < 100:
    ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100000)
    i +=1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpw1w9j_7b/model.ckpt-40
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 40 into /tmp/tmpw1w9j_7b/model.ckpt.
INFO:tensorflow:loss = 66.30565, step = 41
INFO:tensorflow:Saving checkpoints for 50 into /tmp/tmpw1w9j_7b/model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Loss for final step: 66.46154.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpw1w9j_7b/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done r

KeyboardInterrupt: 